In [7]:
import pandas as pd
import numpy as np
import tqdm
from pathlib import Path

# Динамическое получение текущей рабочей директории
current_path = Path.cwd()  # Получает текущую рабочую директорию
root_directory = current_path.parent


df_animals = pd.read_excel(root_directory / "raw_data" / "Датасет на хакатон.xlsx")
df_mutations = pd.read_excel(root_directory / "raw_data" / "Генетические мутации хакатон.xlsx")

In [8]:
root_directory

PosixPath('/Users/holodkovaelizavetaigorevna/Downloads/hakaton')

## Заполнение пустых данных

In [2]:

mask_male = df_animals["Пол"] == "Самец"
mask_female = df_animals["Пол"] == "Самка"

df_animals["Удой л/день"][mask_male] = np.nan

dataset_male = df_animals[df_animals["Пол"] == "Самец"]
dataset_female = df_animals[df_animals["Пол"] == "Самка"]

# Заполнение пропусков для самок по породе и полу
df_animals['Удой л/день'] = df_animals.groupby(['Порода', 'Пол'])['Удой л/день'].transform(lambda x: x.fillna(x.mean()))
df_animals['Прирост веса кг/день'] = df_animals.groupby(['Порода', 'Пол'])['Прирост веса кг/день'].transform(lambda x: x.fillna(x.mean()))
df_animals['Фертильность (%)'] = df_animals.groupby(['Порода', 'Пол'])['Фертильность (%)'].transform(lambda x: x.fillna(x.mean()))


/var/folders/wx/t_ps87396x57bwpc30093rch0000gn/T/ipykernel_49596/4235569056.py:4: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  df_animals["Удой л/день"][mask_male] = np.nan
/var/folders/wx/t_ps87396x57bwpc30093rch0000gn/T/ipykernel_49596/42

In [3]:
# Объединение двух DataFrame по столбцу 'ID_особи'
merged_df = pd.merge(df_animals, df_mutations, on='ID_особи', how='inner')

# Фильтрация df_mutations по ID_особи из df_animals
filtered_mutations = df_mutations[df_mutations['ID_особи'].isin(df_animals['ID_особи'])]
# Фильтруем df_mutations по признакам
relevant_traits = ['Фертильность (%)', 'Прирост веса кг/день', 'Здоровье (1-10)',
       'Коэффициент инбридинга (F)', 'Упитанность', 'Удой л/день',
       'Генетическая ценность (баллы)']
filtered_mutations = df_mutations[df_mutations['Признак'].isin(relevant_traits)]

# Объединяем с df_animals по ID_особи
merged_relevant = pd.merge(df_animals, filtered_mutations, on='ID_особи', how='left')

# Функция для расчёта влияния мутации
def calculate_mutation_effect(row):
    genotype, ref, alt, beta = row["Генотип коровы"], row["ref"], row["alt"], row["beta"]
    if genotype == f"{ref}/{ref}":
        return 1  # Нет влияния
    elif genotype == f"{ref}/{alt}" or genotype == f"{alt}/{ref}":
        return beta / 2  # Половина эффекта
    elif genotype == f"{alt}/{alt}":
        return beta  # Полный эффект
    return np.nan  # Неизвестный генотип

merged_df["mutation_effect"] = merged_df.apply(calculate_mutation_effect, axis=1)

In [4]:
merged_df.to_csv(root_directory / "datasets" / "merged_df.csv", index=False)
df_animals.to_csv(root_directory / "datasets" / "df_animals.csv", index=False)